In [78]:
import tensorflow as tf
import numpy as np

## tensorflow를 활용한 RNN 구축

In [79]:
sample="Mulmi team, cheer-up"
# "Mulmi team, cheer-u" -> "Mulmi team, cheer-up"

In [80]:
idx2char=list(set(sample))
char2idx={c : i for i, c in enumerate(idx2char)}

In [81]:
dic_size=len(char2idx)
hidden_size=num_classes=dic_size
# dic_size:원핫인코딩 크기(rnn input size)
# hidden_size:output size
# num_classes:최종 output size(softmax)
batch_size=1 # 데이터 1개 -> 배치 -> 업데이트
sequence_length=len(sample)-1
learning_rate=0.1

In [82]:
print(hidden_size)
print(num_classes)
print(dic_size)

15
15
15


In [83]:
tf.reset_default_graph()
sample_idx = [char2idx[c] for c in sample]
# sample="Mulmi team, cheer-up"
xdata=[sample_idx[:-1]] # 맨 마지막 숫자가 빠지고 출력
ydata=[sample_idx[1:]] # 처음 숫자가 빠지고 끝까지 출력
x=tf.placeholder(tf.int32,[None,sequence_length])
y=tf.placeholder(tf.int32,[None,sequence_length])
x_one_hot=tf.one_hot(x,num_classes)
cell=tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
ini_state=cell.zero_state(batch_size, tf.float32)
outputs, _state=tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=ini_state, dtype=tf.float32)

In [84]:
xfc=tf.reshape(outputs,[-1,hidden_size])
outputs=tf.contrib.layers.fully_connected(xfc,num_classes, activation_fn=None)
outputs=tf.reshape(outputs,[batch_size,sequence_length, num_classes])
weights=tf.ones([batch_size, sequence_length])
sequence_loss=tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=y, weights=weights)
loss=tf.reduce_mean(sequence_loss)
train=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction=tf.argmax(outputs,axis=2)

In [85]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        cv, _=sess.run([sequence_loss, train],feed_dict={x:xdata, y:ydata})
        res=sess.run(prediction, feed_dict={x:xdata})
        res_str=[idx2char[c] for c in np.squeeze(res)]
        print(i,"loss:",cv,"prediction:","".join(res_str))

0 loss: 2.7089214 prediction: umm          euuuuu
1 loss: 2.5229628 prediction: umm  ueuuuuuuuuuuuu
2 loss: 2.3509436 prediction: mmm  taau  eeeeeeuu
3 loss: 2.0516894 prediction: mmm  teae  eeeee-up
4 loss: 1.6926514 prediction: mmm  team  ceeee-up
5 loss: 1.3118911 prediction: mmmi team  cheee-up
6 loss: 0.9854412 prediction: mmmi team, cheer-up
7 loss: 0.7207087 prediction: ummi team, cheer-up
8 loss: 0.52459997 prediction: ulmi team, cheer-up
9 loss: 0.35374856 prediction: ulmi team, cheer-up
10 loss: 0.24132329 prediction: ulmi team, cheer-up
11 loss: 0.1525188 prediction: ulmi team, cheer-up
12 loss: 0.10293911 prediction: ulmi team, cheer-up
13 loss: 0.071052 prediction: ulmi team, cheer-up
14 loss: 0.050959736 prediction: ulmi team, cheer-up
15 loss: 0.035228293 prediction: ulmi team, cheer-up
16 loss: 0.025361842 prediction: ulmi team, cheer-up
17 loss: 0.01975616 prediction: ulmi team, cheer-up
18 loss: 0.014721804 prediction: ulmi team, cheer-up
19 loss: 0.011365376 predicti

In [61]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

dataset=[["사과","치즈","생수"],
        ["생수","호두","치즈","고등어"],
        ["수박","사과","생수"],
        ["생수","호두","치즈","옥수수"]]

In [63]:
te=TransactionEncoder()
te_ary=te.fit(dataset).transform(dataset)
te_ary

array([[False,  True,  True, False, False,  True, False],
       [ True, False,  True, False, False,  True,  True],
       [False,  True,  True,  True, False, False, False],
       [False, False,  True, False,  True,  True,  True]])

In [68]:
df=pd.DataFrame(te_ary, columns=te.columns_)
df
freq_item=apriori(df,min_support=0.5, use_colnames=True)
freq_item

,support,itemsets
0,0.50,(사과)
1,1.00,(생수)
2,0.75,(치즈)
3,0.50,(호두)
4,0.50,"(생수, 사과)"
5,0.75,"(생수, 치즈)"
6,0.50,"(생수, 호두)"
7,0.50,"(호두, 치즈)"
8,0.50,"(생수, 호두, 치즈)"


In [86]:
from mlxtend.frequent_patterns import association_rules
association_rules(freq_item,metric="lift", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(생수),(사과),1.00,0.50,0.50,0.500000,1.000000,0.000,1.0
1,(사과),(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf
2,(생수),(치즈),1.00,0.75,0.75,0.750000,1.000000,0.000,1.0
3,(치즈),(생수),0.75,1.00,0.75,1.000000,1.000000,0.000,inf
4,(생수),(호두),1.00,0.50,0.50,0.500000,1.000000,0.000,1.0
5,(호두),(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf
6,(호두),(치즈),0.50,0.75,0.50,1.000000,1.333333,0.125,inf
7,(치즈),(호두),0.75,0.50,0.50,0.666667,1.333333,0.125,1.5
8,"(생수, 호두)",(치즈),0.50,0.75,0.50,1.000000,1.333333,0.125,inf
9,"(생수, 치즈)",(호두),0.75,0.50,0.50,0.666667,1.333333,0.125,1.5
